In [1]:
from collection import SparseCollection, SparseCollectionCSR
from backend import TYPE
import json
import psutil
from text2vec import BagOfWords
import torch
from collections import defaultdict
from tqdm import tqdm

import pyterrier  as pt
import pandas as pd
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
indexref = pt.IndexRef.of("../syn-question-col-analysis/datasets/msmarco/terrier_index/")
index = pt.IndexFactory.of(indexref)

def tp_func():
    stops = pt.autoclass("org.terrier.terms.Stopwords")(None)
    stemmer = pt.autoclass("org.terrier.terms.PorterStemmer")(None)
    def _apply_func(row):
        words = row["query"].split(" ") # this is safe following pt.rewrite.tokenise()
        words = [stemmer.stem(w) for w in words if not stops.isStopword(w) ]
        return words
    return _apply_func 

pipe = pt.rewrite.tokenise() >> pt.apply.query(tp_func())
token2id = {word.getKey():i for i,word in enumerate(index.getLexicon()) }

vocab_size = len(index.getLexicon())

def tokenizer(text):
    tokens_ids = []
    for token in pipe(pd.DataFrame([{"qid":0, "query":text.lower()}]))["query"][0]:
        if token in token2id:
            token_id=token2id[token]
            if token_id is not None:
                tokens_ids.append(token_id)
    return tokens_ids

bow = BagOfWords(tokenizer, vocab_size)

10:37:23.660 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [3]:
sparse_collection = SparseCollectionCSR.load_from_file("beir_datasets/msmarco/csr_anserini_bm25_12_075")


In [4]:

csr_matrix_cpu = torch.sparse_csr_tensor(*sparse_collection.sparse_vecs, sparse_collection.shape)


/tmp/ipykernel_2191770/2760414820.py:1: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  csr_matrix_cpu = torch.sparse_csr_tensor(*sparse_collection.sparse_vecs, sparse_collection.shape)


In [1]:
from pytorch_quantization import tensor_quant

ImportError: /lib/x86_64-linux-gnu/libc.so.6: version `GLIBC_2.32' not found (required by /home/tiagoalmeida/.local/lib/python3.10/site-packages/pytorch_quantization/cuda_ext.cpython-310-x86_64-linux-gnu.so)

In [5]:
torch.__version__

'2.1.0+cu121'

In [ ]:
csr_matrix_gpu = csr_matrix_cpu.to("cuda:0")

In [15]:
with open("../syn-question-col-analysis/question_generation/gen_output/msmarco/selected_corpus_lm_fcm_STD2_L10000_gpt-neo-1.3B_BS_5_E13931.459746599197.jsonl") as f:
    questions = [line for line in map(json.loads, f)]

In [16]:
def text_to_dense_torch(text):
    b = bow(text)
    return torch.sparse_coo_tensor([list(b.keys())], list(b.values()), (vocab_size,), dtype=torch.float32).to_dense()

def text_to_sparse_torch(text):
    b = bow(text)
    return torch.sparse_coo_tensor([list(b.keys()), [0]*len(b)], list(b.values()), (vocab_size,1), dtype=torch.float32)#.to_sparse_csc()


def retrieve_topk_gpu(question, at=10):
    query_gpu = text_to_dense_torch(question).to("cuda:1")
    r = torch.topk(csr_matrix_gpu @ query_gpu, k=at, dim=0).indices#.cpu()
    #print("tokenizer stack", end_stack-end_tok_t, "retrieve",end_r_t-end_stack)
    return r

def retrieve_topk_cpu(question, at=10):
    query = text_to_dense_torch(question)
    r = torch.topk(csr_matrix_cpu @ query, k=at, dim=0).indices.cpu()
    #print("tokenizer stack", end_stack-end_tok_t, "retrieve",end_r_t-end_stack)
    return r


def batch_retrieve_topk(questions_text, at=10):
    # does not work!
    queries_gpu_tensor = torch.zeros(vocab_size, len(questions_text))
    end_tok_t = time.time()
    for i, question in enumerate(questions_text):
        queries_gpu_tensor[:,i] = text_to_dense_torch(question)

    queries_gpu_tensor = queries_gpu_tensor.to("cuda:1")

    end_stack = time.time()
    r = torch.topk(csr_matrix_gpu @ queries_gpu_tensor, k=at, dim=0).indices.T.cpu()
    end_r_t = time.time()
    #print("tokenizer stack", end_stack-end_tok_t, "retrieve",end_r_t-end_stack)
    return r

In [17]:
import time

results = []
start_t = time.time()
for question in tqdm(questions):
    results.append(retrieve_topk_gpu(question["question"], at=10))
end_t = time.time()

#print(1000/(end_t-start_t), "it/s")

  0%|          | 0/47931 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:1 and cuda:0! (when checking argument for argument mat in method wrapper_SparseCsrCUDA_out_addmv_out)

In [18]:
# batch retrievel
batches_questions = [list(map(lambda x:x["question"], questions[i:i+64])) for i in range(64,len(questions),64)]

results = []
for bq in tqdm(batches_questions):
    results.append(batch_retrieve_topk(bq,10))


  0%|          | 0/748 [00:00<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cuda:1! (when checking argument for argument mat2 in method wrapper_SparseCsrCUDA__mm)

In [14]:

batches_questions

[]

In [19]:
query_gpu = text_to_dense_torch(questions[0]["question"]).to("cuda:0")
r = torch.topk(csr_matrix_gpu @ query_gpu, k=10, dim=0)

In [20]:
r

torch.return_types.topk(
values=tensor([73.5256, 16.0785, 15.6707, 13.2659, 12.6108, 12.5160, 12.4417, 12.3735,
        12.0173, 11.8916], device='cuda:0'),
indices=tensor([ 341, 9927,   44, 9688, 4810, 3132, 1989, 1985, 4819, 3131],
       device='cuda:0'))

In [21]:
sparse_collection.shape

(10000, 1170682)

In [26]:
bow(questions[0]["question"])

defaultdict(float,
            {880134: 1.0, 869173: 1.0, 274027: 1.0, 541942: 1.0, 652667: 1.0})

In [24]:
query_tokens = pipe(pd.DataFrame([{"qid":0, "query":questions[0]["question"].lower()}]))["query"][0]
query_tokens

['relationship', 'rarotongan', 'cook', 'island', 'maori']

In [29]:
PATH_TO_MSMARCO = "../syn-question-col-analysis/datasets/msmarco/corpus_L8841823.jsonl"

def get_title_abstract(string):
    data = json.loads(string)
    title, abstract = data["title"], data["abstract"]
    return f"{title} {abstract}"

with open(PATH_TO_MSMARCO) as f:
    for doc in map(get_title_abstract, f):
       
        break

In [31]:
bow(doc)

defaultdict(float,
            {839186: 1.0,
             266051: 1.0,
             927074: 2.0,
             684233: 1.0,
             373151: 1.0,
             523132: 1.0,
             998799: 2.0,
             651144: 1.0,
             845056: 1.0,
             533255: 1.0,
             767761: 1.0,
             257234: 1.0,
             474305: 1.0,
             523347: 1.0,
             77832: 1.0,
             140132: 1.0,
             884129: 1.0,
             367894: 1.0,
             1066913: 1.0,
             666305: 1.0,
             506868: 1.0,
             1042085: 1.0,
             530548: 1.0,
             625474: 1.0,
             754559: 1.0})

In [ ]:
batch_size = 2
batched_queries = [questions[i:i + batch_size] for i in range(0, len(questions), batch_size)]

results = []
start_t = time.time()
for batch in tqdm(batched_queries):
    results.append(batch_retrieve_topk(list(map(lambda x:x["question"], batch))))
end_t = time.time()

print("took", end_t-start_t)
# 0.003 - 0.003
# 0.008 - 0.02

In [9]:
query_gpu = text_to_dense_torch(question["question"]).to("cuda:1")
end_stack = time.time()
r = torch.topk(csr_matrix_gpu @ query_gpu, k=10, dim=0).indices

In [10]:
bow()

tensor([1424203, 2678308, 3748774, 3233037, 2741266, 5938529, 5026090, 5495619,
        2542788,  838903], device='cuda:1')

<function Tensor.to_sparse_csc>

In [6]:
def text_to_dense_torch(text, dim):
    b = bow(text)
    return torch.sparse_coo_tensor([list(b.keys())], list(b.values()), (dim,), dtype=torch.float32).to_dense()


In [7]:
csr_matrix_gpu = csr_matrix_cpu.to("cuda:1")


In [8]:
PATH_TO_QUESTIONS = "../syn-question-col-analysis/datasets/msmarco/relevant_pairs.jsonl"


In [20]:

with open(PATH_TO_QUESTIONS) as f:
    questions = {line["question"] for line in map(json.loads,f)}

questions = [text_to_dense_torch(q, csr_matrix_cpu.shape[1]) for q in questions]
queries_gpu = torch.stack(questions[:20], -1).to("cuda:1")

#sort, idx = (csr_matrix_gpu @ queries_gpu).sort(descending=True, dim=0)
#result = idx[:10].cpu()
result = torch.topk(csr_matrix_gpu @ queries_gpu, k=10, dim=0).indices.T.cpu()

In [22]:
result

tensor([[8617271, 7607669, 5466810, 1379245, 1379240, 5466807, 1664523, 8617274,
          547444,  269428],
        [1929910, 3572695, 7839904, 1288938, 4842897, 2507917, 7839906, 3572702,
         5359212, 3937200],
        [2533260, 5012351, 8121380,  719552, 5291683, 7952865, 5291686,  719550,
         6528714, 7088568],
        [8049577, 1433123,  669004, 4563960, 1584254, 3410067, 2055598, 8049578,
           16845, 3552218],
        [8635981, 7267248,  527698, 3260688, 1837110, 1958102, 1958100, 7267243,
         8199361, 7367407],
        [8760867, 8760864, 3641634, 2787508, 4788864, 2157456, 8760868, 8760873,
         3620983, 3342992],
        [7778351, 2868845, 6436703, 7778348, 4164404, 4337532, 2197526, 2997653,
         7670593, 2160853],
        [7447941, 8433858, 6654655, 8433854, 7896211, 2747492, 2365660, 5638740,
         4704978, 2702419],
        [8160520, 3838645,  554521, 4511137,  398442, 4575877, 5218014, 8160527,
         1901881, 8478604],
        [ 536176, 3

In [13]:
del queries_gpu
del result

In [13]:
import time

time_list = []
for _ in range(20):
    
    with open(PATH_TO_QUESTIONS) as f:
        questions = {line["question"] for line in map(json.loads,f)}

    questions = [text_to_dense_torch(q, csr_matrix_cpu.shape[1]) for q in questions]
    queries_gpu = torch.stack(questions, -1).to("cuda:1")

    start_t = time.time()
    #sort, idx = (csr_matrix_gpu @ queries_gpu).sort(descending=True, dim=0)
    #result = idx[:10].cpu()
    result = torch.topk(csr_matrix_gpu @ queries_gpu, k=100000, dim=0).indices.cpu()
    time_list.append(time.time()-start_t)
    
    for q in questions:
        del q
    del questions
    del queries_gpu
    del result 
    #del sort
    #del idx

In [14]:
time_list

[0.4787881374359131,
 0.44898533821105957,
 0.4485352039337158,
 0.4484899044036865,
 0.44817113876342773,
 0.4484434127807617,
 0.4484429359436035,
 0.44898128509521484,
 0.44836854934692383,
 0.44843435287475586,
 0.4483344554901123,
 0.4485807418823242,
 0.44898390769958496,
 0.44857192039489746,
 0.44869017601013184,
 0.448455810546875,
 0.44840002059936523,
 0.4485960006713867,
 0.4486415386199951,
 0.4485054016113281]

In [46]:
csr_matrix_gpu = csr_matrix_cpu.to("cuda:1")

In [68]:
queries = [text_to_dense_torch("what is the meaning of life?", csr_matrix_cpu.shape[1]),
           text_to_dense_torch("what time it is?", csr_matrix_cpu.shape[1]),
            ]

queries = torch.stack(queries, -1)
queries_gpu = queries.to("cuda:1")
    


tensor([[4.3629, 3.6962],
        [0.5774, 0.0000],
        [0.5461, 1.5242],
        ...,
        [0.5942, 0.0000],
        [0.5746, 2.5969],
        [1.1826, 0.5707]], device='cuda:1')

In [79]:
import time
time_list = []
for _ in range(20):
    
    queries = [text_to_dense_torch("what is the meaning of life?", csr_matrix_cpu.shape[1]),
           text_to_dense_torch("what time it is?", csr_matrix_cpu.shape[1]),
           text_to_dense_torch("what time it is? 2", csr_matrix_cpu.shape[1]),
           text_to_dense_torch("wagfsdf asdg asg ?", csr_matrix_cpu.shape[1]),
           #text_to_dense_torch("wwef faes ewq ta afewf s?", csr_matrix_cpu.shape[1]),
            ]

    queries = torch.stack(queries, -1)
    queries_gpu = queries.to("cuda:1")

    start_t = time.time()
    csr_matrix_gpu @ queries_gpu
    time_list.append(time.time()-start_t)

In [80]:
sum(time_list)/len(time_list)*1000


0.08742809295654297

In [88]:

with open(PATH_TO_QUESTIONS) as f:
    questions = {line["question"] for line in map(json.loads,f)}


In [91]:
queries_gpu = queries.to("cuda:1")

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.42 GiB (GPU 1; 7.79 GiB total capacity; 6.26 GiB already allocated; 1.19 GiB free; 6.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [94]:
sum(time_list)/len(time_list)*1000

0.1820206642150879

In [ ]:

%%timeit -n 1000
retrieve_gpu_nvprims_nvfuser(query_gpu)

In [47]:
query_gpu = query.to("cuda:1")
retrieve_gpu4(csr_matrix_gpu, query_gpu)

tensor([4.3629, 0.5774, 0.5461,  ..., 0.5942, 0.5746, 1.1826], device='cuda:1')

In [48]:
%%timeit -l 1000
retrieve_gpu4(csr_matrix_gpu, query_gpu)

11.1 ms ± 78.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [44]:
%%timeit
csr_matrix @ query

11.1 ms ± 88 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [53]:
%%timeit
torch.topk(csr_matrix@query,k=10)

190 ms ± 4.17 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


torch.Size([30522])

tensor(crow_indices=tensor([     0,     40,     67,  ..., 465841, 465881,
                            465917]),
       col_indices=tensor([ 1012,  1025,  1996,  ...,  9949, 17502, 26110]),
       values=tensor([2., 1., 6.,  ..., 1., 1., 3.]), size=(10000, 30522),
       nnz=465917, layout=torch.sparse_csr)

In [15]:
sparseCSR_collection.transform(BM25Transform())

Converting to BM25 matrix: 100%|██████████| 10000/10000 [00:00<00:00, 27992.58it/s]


In [16]:
sparseCSR_collection.sparse_vecs

(tensor([     0,     40,     67,  ..., 465841, 465881, 465917],
        dtype=torch.int32),
 tensor([ 1012,  1025,  1996,  ...,  9949, 17502, 26110], dtype=torch.int32),
 tensor([0.0000, 2.1513, 0.0000,  ..., 6.4540, 8.6054, 9.9709]))

In [17]:
torch.sparse_csr_tensor(*sparseCSR_collection.sparse_vecs, sparseCSR_collection.shape)

tensor(crow_indices=tensor([     0,     40,     67,  ..., 465841, 465881,
                            465917]),
       col_indices=tensor([ 1012,  1025,  1996,  ...,  9949, 17502, 26110]),
       values=tensor([0.0000, 2.1513, 0.0000,  ..., 6.4540, 8.6054, 9.9709]),
       size=(10000, 30522), nnz=465917, layout=torch.sparse_csr)